In [4]:
import configparser
import numpy as np
import pandas as pd
import tasks.models as m

from sqlalchemy import create_engine, func, distinct
from sqlalchemy.orm import Session
import redis

In [5]:
config = configparser.ConfigParser()
config.read("config/config.ini")

engine = create_engine(config["DB"].get("db_url"))
session = Session(engine)

r = redis.Redis(
    host=config["REDIS"].get("host"),
    port=config["REDIS"].get("port"),
    db=config["REDIS"].get("db"),
)

In [6]:
s = (
    session.query(m.Redditor_Subreddit_Comment.redditor_id)
    .group_by(m.Redditor_Subreddit_Comment.redditor_id)
    .having(func.count(m.Redditor_Subreddit_Comment.subreddit_id.distinct()) > 2)
    .subquery()
)
redditor_subreddit_comment = np.array(
    session.query(m.Redditor_Subreddit_Comment.redditor_id, m.Subreddit.name)
    .filter(m.Redditor_Subreddit_Comment.redditor_id.in_(s))
    .join(m.Subreddit, m.Redditor_Subreddit_Comment.subreddit_id == m.Subreddit.id)
    .all()
).T
df = pd.DataFrame(
    {
        "userID": redditor_subreddit_comment[0],
        "subredditID": redditor_subreddit_comment[1],
        "rating": np.ones_like(redditor_subreddit_comment[0]),
    }
)
df["rating"] = pd.to_numeric(df["rating"])

,userID,subredditID,rating
0,101stexi,guns,1
1,101stexi,MTB,1
2,101stexi,crossfit,1
3,102kp8,PropagandaPosters,1
4,102kp8,PropagandaPosters,1
...,...,...,...
31653,zx4mnmk,Supplements,1
31654,zx4mnmk,Nootropics,1
31655,zx4mnmk,Nootropics,1
31656,zx4mnmk,Supplements,1


In [13]:
user_ids = np.unique(df['userID'])

In [17]:
def compare_recommendation_and_interaction(user_id):
    print(np.unique(df[df['userID']==user_id]['subredditID']))
    print(r.lrange(user_id, 0, -1))

In [18]:
compare_recommendation_and_interaction(user_ids[0])

['MTB' 'crossfit' 'guns']
[b'weedstocks', b'vinyl', b'Coffee', b'TikTokCringe', b'stopdrinking']


In [19]:
compare_recommendation_and_interaction(user_ids[1])

['CitiesSkylines' 'PropagandaPosters' 'Warhammer40k' 'hoi4']
[b'PoliticalCompassMemes', b'vexillology', b'eu4', b'Warhammer', b'Warthunder']


In [20]:
compare_recommendation_and_interaction(user_ids[2])

['linuxquestions' 'makinghiphop' 'unixporn']
[b'linuxmasterrace', b'linux', b'Piracy', b'linux_gaming', b'archlinux']


In [21]:
compare_recommendation_and_interaction(user_ids[3])

['Instantregret' 'apolloapp' 'fatlogic' 'hometheater']
[b'DeadorVegetable', b'InteriorDesign', b'XboxSeriesX', b'tooktoomuch', b'diablo3']
